# Comparação de modelos - 1° gol

## Sumário
1. [Introdução](#c1)
2. [Apresentação dos modelos](#c2)
3. [Comparação de resultados e métricas](#c3)
4. [Conclusões](#c4)

## <a name="c1"></a>1. Introdução

Ao longo do projeto desenvolvido em parceria com a IBM, foram constuídos diversos modelos de inteligência artificial que objetivam a correta predição de resultados em partidas de futebol. Dentre essas predições, encontra-se a determinação de qual time, dados dois adversários, será o primeiro a pontuar em um embate. Nesse contexto, adotou-se uma abordagem baseada na criação de modelos com funcionamentos distintos que buscassem responder a mesma pergunta, seguida da comparação entre esses diferentes modelos preditivos.

Tal prática tem se mostrado essencial para alcançar previsões mais precisas e eficientes, permitindo avaliar a performance de diferentes algoritmos, impacto do uso de determinadas variáveis do conjunto de dados nos resultados, e, por conseguinte, qual técnica se adequa melhor ao projeto em questão.

Neste trabalho, realizaremos uma análise comparativa entre dois modelos de IA, aplicados à predição do primeiro gol em partidas de futebol. Através dessa comparação, será possível explorar as diferenças em termos de precisão, eficiência, vantagens e desvantagens dos modelos.

Os modelos aqui descritos são compostos da junção de duas ou mais técnicas e algoritmos para análise de dados, incluindo mas não se limitando a técnicas para agrupamento (clusterização), classificação e redução de dimensionalidade. Por fim, ambos os modelos analisados estão disponíveis neste repositório, podendo ser consultados individualmente.


## <a name="c2"></a> 2. Apresentação dos modelos

### 2.1. Modelo 1 - K-means e probabilidade empírica

O K-Means é um algoritmo de aprendizado não supervisionado que agrupa dados em "K" clusters com base em suas características. Ele funciona atribuindo cada ponto de dado ao cluster mais próximo (com base na distância euclidiana), recalculando as médias dos clusters e repetindo até que as atribuições não mudem mais. [[1]](https://docs.aws.amazon.com/pt_br/sagemaker/latest/dg/algo-kmeans-tech-notes.html#:~:text=O%20k%2Dmeans%20%C3%A9%20um,n%C3%BAmero%20de%20atributos%20da%20observa%C3%A7%C3%A3o)

Segue o passo a passo sobre o funcionamento do modelo e suas métricas.

In [ ]:
# Import das bibliotecas que serão utilizadas no modelo preditivo
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
import numpy as np
from sklearn.decomposition import PCA
from sklearn.impute import SimpleImputer
from sklearn.cluster import KMeans
from sklearn.metrics import confusion_matrix, accuracy_score, precision_recall_curve, roc_curve, auc, make_scorer, silhouette_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import label_binarize
from sklearn.svm import SVC
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Leitura dos dados e merge dos dados de times, transformando-os em um único dataframe.

In [ ]:
matches = pd.read_csv('/content/drive/Shareddrives/Drive IBMatch/Data/brazil-serie-a-matches-2024-to-2024-stats (6).csv')
times1 = pd.read_csv('/content/drive/Shareddrives/Drive IBMatch/Data/brazil-serie-a-teams-2024-to-2024-stats (4).csv')
times2 = pd.read_csv('/content/drive/Shareddrives/Drive IBMatch/Data/brazil-serie-a-teams2-2024-to-2024-stats (2).csv')

teams = pd.merge(times1, times2, on="common_name", how="left")

teams.insert(0, "team_index", list(range(1, 21)))

Limpeza de dados do dataframe de partidas, visando reduzir o dataframe apenas a tabelas importantes, além de remover dados vazios e partidas suspensas, canceladas e incompletas.

In [ ]:
categorical_matches = pd.DataFrame(matches)
categorical_matches = categorical_matches[~categorical_matches['status'].isin(['suspended', 'canceled', 'incomplete'])]
#categorical_matches = categorical_matches[~categorical_matches['first_to_score'].isin(['N/A'])]

matches = matches[["status", "home_team_name", "away_team_name", "first_to_score"]]
matches = matches[~matches['status'].isin(['suspended', 'canceled', 'incomplete'])]
matches = matches[~matches['first_to_score'].isin(['N/A'])]

In [ ]:
teams_categorical = teams
teams = teams.select_dtypes(include=['int64', 'float64'])

In [ ]:
# Limpeza com imputer
imputer_numbers = SimpleImputer(missing_values=np.nan, strategy="median")
imputer_numbers.fit(teams)

# imputer_moda = SimpleImputer(missing_values=np.nan, strategy="most_frequent")
# imputer_moda.fit(data_categorical)

In [ ]:
selected_tables = [
    "wins",
    "draws",
    "losses",
    "first_team_to_score_count",
    "goals_scored",
    "goals_conceded",
    "clean_sheets",
    "btts_count",
    "fts_count",
]

train_data = pd.DataFrame(teams[selected_tables])

selected_tables.insert(1, "common_name")

data = pd.DataFrame(teams)
data["common_name"] = teams_categorical["common_name"]

Utilizando Grid Search, são testadas diversas configurações para o K-Means, visando encontrar a melhor configuração do modelo. Para chegar ao resultado do melhor modelo, é utiliza o índice da silhueta.

In [ ]:
def silhouette_scorer(estimator, data):
    labels = estimator.fit_predict(data)
    return silhouette_score(data, labels)

kmeans = KMeans()

param_grid = {
    'n_clusters': [3],
    'n_init': [10, 20, 30],
    'max_iter': [300, 500],
    'tol': [1e-4, 1e-3]
}

grid_search = GridSearchCV(estimator=kmeans, param_grid=param_grid, scoring=silhouette_scorer, cv=3)
grid_search.fit(teams)

Definição do algoritmo K-Means, passando as melhores configurações definidas pelos hiperparâmetros.

In [ ]:
num_clusters = grid_search.best_params_['n_clusters']
max_iter = grid_search.best_params_['max_iter']
n_init = grid_search.best_params_['n_init']
tol = grid_search.best_params_['tol']
kmeans = KMeans(n_clusters=num_clusters, max_iter=max_iter, n_init=n_init, tol=tol)
kmeans.fit(train_data)

In [ ]:
centroids = kmeans.cluster_centers_
labels = kmeans.labels_

Plotagem de um gráfico de barras, visando demonstrar os pesos de cada feature nas previsões do modelo.

In [ ]:
variable_weights = pd.DataFrame(dict(zip(train_data.columns, kmeans.cluster_centers_[labels])))
variable_weights = variable_weights.mean().sort_values()

plt.figure(figsize=(10, 6))
plt.barh(variable_weights.index, variable_weights.values, color='skyblue')
plt.xlabel("Média dos Pesos")
plt.title("Variáveis Ordenadas pelos Maiores Pesos")
plt.show()

matches = pd.DataFrame(matches)

O K-Means teve a responsabilidade de separar cada time participante do Brasilerão em três categorias, sendo elas, times pequenos, times médios e times grandes. A partir dessa separação é possível calcular a probabilidade de cada time fazer o primeiro gol em uma partida. Abaixo, gráficos gerados a partir do uso do algortimo K-means para clusterização.

In [ ]:
plt.scatter(data[labels == 2]['first_team_to_score_count'], data[labels == 2]['clean_sheets'], color='red', label='Grandes')
plt.scatter(data[labels == 1]['first_team_to_score_count'], data[labels == 1]['clean_sheets'], color='blue', label='Médio')
plt.scatter(data[labels == 0]['first_team_to_score_count'], data[labels == 0]['clean_sheets'], color='green', label='Pequeno')
plt.scatter(centroids[:, 0], centroids[:, 1], color='black', marker='x', label='Centroids')
plt.xlabel('Wins')
plt.ylabel('Losses')
plt.title('Clusters dos times')
plt.legend()
plt.show()

fig = px.scatter_3d(train_data, x='first_team_to_score_count', y='fts_count', z='clean_sheets', color='first_team_to_score_count', title='Clusters dos times', labels={
        'first_team_to_score_count': 'Primeiro Time a Marcar',
        'fts_count': 'FTS',
        'clean_sheets': 'Jogos Sem Sofrer Gol'
    })
fig.add_scatter3d(x=centroids[:, 0], y=centroids[:, 1], z=centroids[:, 2], mode='markers', marker=dict(color='black', size=2, symbol='x'), name='Centroids')
fig.show()

Criação do DataFrame de resultados do modelo, e os times, que antes estavam separados entre os clusters 0, 1 e 2, agora foram separados de forma categórica, indicando os grupos através dos títulos Pequeno, Médio e Grande.

In [ ]:
results_km = pd.DataFrame()
results_km["team_index"] = data["team_index"]
results_km["team_name"] = data["common_name"]
results_km["labels_classif"] = labels

classificacao = list()
for i in range(0, 20):
    if results_km["labels_classif"][i] == 0:
        classificacao.append("Grande")
    elif results_km["labels_classif"][i] == 1:
        classificacao.append("Médio")
    elif results_km["labels_classif"][i] == 2:
        classificacao.append("Pequeno")

results_km["classificacao"] = classificacao

In [ ]:
times_classificados = {
    "Pequeno": list(),
    "Médio": list(),
    "Grande": list()
}

index = 0
for key, value in results_km["classificacao"].items():
    if value == "Pequeno":
        times_classificados["Pequeno"].append(results_km["team_name"][index])
    elif value == "Médio":
        times_classificados["Médio"].append(results_km["team_name"][index])
    elif value == "Grande":
        times_classificados["Grande"].append(results_km["team_name"][index])
    index += 1

Para calcular a probabilidade de cada time fazer o primeiro gol, foi utilizada a seguinte fórmula matemática:

$P(1° gol) = \frac{TAxX}{Qpm}$

- TAxX: O TAxX é o total de 1°s gols do time $A$ contra times do grupo X, sendo o grupo X o grupo ao qual o time B pertence.
- Qpm: Quantidade de partidas que o time A disputou contra times do grupo X, sendo o grupo $X$ o grupo ao qual o time B pertence.

In [ ]:
def calcular_probabilidade(time_a, time_b):
    grupo_time_b = list()
    for grupo in times_classificados.values():
        if grupo.__contains__(time_b):
            grupo_time_b = grupo

    a_team_matches = matches.query(f"home_team_name == '{time_a}' or away_team_name == '{time_a}'")
    a_team_matches_against_group_b = a_team_matches[a_team_matches['away_team_name'].isin(grupo_time_b) | a_team_matches['home_team_name'].isin(grupo_time_b)]

    TAxX = a_team_matches_against_group_b[a_team_matches_against_group_b['first_to_score'] == time_a].shape[0]

    Qpm = len(a_team_matches_against_group_b)

    probabilidade = TAxX / Qpm if TAxX > 0 and Qpm > 0 else 0.5

    return probabilidade

Considerando que a função `calcular_probabilidade()` retorna a probabilidade de cada time fazer o primeiro gol, a função `comparar_times()` retorna o nome que tem a maior probabilidade de fazer o primeiro gol.

In [ ]:
def comparar_times(time_a, time_b):
    prob_a = calcular_probabilidade(time_a, time_b)
    prob_b = calcular_probabilidade(time_b, time_a)

    soma_probabilidades = prob_a + prob_b
    prob_a_normalizada = (prob_a / soma_probabilidades) * 100
    prob_b_normalizada = (prob_b / soma_probabilidades) * 100

    #print(f"{time_a} - ({prob_a_normalizada:.1f}%) x ({prob_b_normalizada:.1f}%) - {time_b}")

    if prob_a_normalizada > prob_b_normalizada:
        return time_a
    elif prob_b_normalizada > prob_a_normalizada:
        return time_b
    else:
        return "Empate"

In [ ]:
def prever_primeiro_gol(row):
    time_a = row['home_team_name']
    time_b = row['away_team_name']
    resultado = comparar_times(time_a, time_b)

    if time_a in resultado:
        return time_a
    elif time_b in resultado:
        return time_b
    else:
        return 'Empate'

matches['previsao_primeiro_gol'] = matches.apply(prever_primeiro_gol, axis=1)

matches = matches.dropna(subset=['previsao_primeiro_gol', 'first_to_score'])

matches['previsao_primeiro_gol'] = matches['previsao_primeiro_gol'].astype(str)
matches['first_to_score'] = matches['first_to_score'].astype(str)

y_true = matches['first_to_score']
y_pred = matches['previsao_primeiro_gol']

Para demonstrar o resultado do modelo, as seguintes métricas são definidas:

- Matriz de confusão
- Cálculo da acurácia
- Total de erros e acertos
- Curva de precisão e recall
- Indíce da silhueta
- Através dessas métricas, é possível avaliar a qualidade do modelo, assim como permite identificar onde ele deve melhorar.

Plotagem da matriz de confusão, além de mostrar métricas relacionadas a acurácio do modelo e quantidade total de acertos e erros.

In [ ]:
matriz_confusao = confusion_matrix(y_true, y_pred, labels=matches['first_to_score'].unique())

plt.figure(figsize=(10,7))
sns.heatmap(matriz_confusao, annot=True, fmt='d', cmap='Blues',
            xticklabels=matches['first_to_score'].unique(),
            yticklabels=matches['first_to_score'].unique())

plt.title('Matriz de Confusão - Previsão vs Realidade')
plt.xlabel('Times Previstos')
plt.ylabel('Times Reais')
plt.show()

acertos = matriz_confusao.diagonal().sum()
erros = matriz_confusao.sum() - acertos
print(f"Total de acertos: {acertos}")
print(f"Total de erros: {erros}")

acuracia = accuracy_score(y_true, y_pred)
print(f"Acurácia do modelo: {(acuracia * 100):.2f}%")


Plota um Gráfico que visa demonstra a curva de precisão e recall

In [ ]:
acertos = matriz_confusao.diagonal().sum()
erros = matriz_confusao.sum() - acertos
print(f"Total de acertos: {acertos}")
print(f"Total de erros: {erros}")

acuracia = accuracy_score(y_true, y_pred)
print(f"Acurácia do modelo: {(acuracia * 100):.2f}%")

classes = matches['first_to_score'].unique()
y_true_bin = label_binarize(y_true, classes=classes)
y_pred_bin = label_binarize(y_pred, classes=classes)

precision = dict()
recall = dict()
for i in range(len(classes)):
    precision[i], recall[i], _ = precision_recall_curve(y_true_bin[:, i], y_pred_bin[:, i])

plt.figure(figsize=(8, 6))
for i in range(len(classes)):
    plt.plot(recall[i], precision[i], marker='.', label=classes[i])

plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Curva de Precisão-Recall para cada time')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Após treinar o modelo K-Means
inertia = kmeans.inertia_
print(f"Inércia: {inertia}")

Gráfico representando o Índice da Silhueta e o Método do Cotovelo.

In [ ]:
from sklearn.metrics import silhouette_score

# Calculando o índice de silhueta
silhouette_avg = silhouette_score(teams, kmeans.labels_)
print(f"Índice de Silhueta: {silhouette_avg}")

import matplotlib.pyplot as plt
inertia_values = []

# Testando diferentes números de clusters
for n_clusters in range(1, 11):
    kmeans = KMeans(n_clusters=n_clusters)
    kmeans.fit(teams)
    inertia_values.append(kmeans.inertia_)

plt.plot(range(1, 11), inertia_values, marker='o')
plt.xlabel('Número de Clusters')
plt.ylabel('Inércia')
plt.title('Método do Cotovelo')
plt.show()

### 2.2. Modelo 2 - K-means e probabilidade empírica

O segundo modelo analisado utiliza-se do algoritmo de clusterização K-means para agrupar os diferentes times em grupos e, a partir disso, compara-se o desempenho de um determinado time contra times que façam parte do grupo do time adversário. Dessa forma, assumindo que times que façam parte do mesmo cluster têm comportamento e desempenho semelhante, somos capazes de prever a probabilidade de cada um dos times possui de ser o primeiro a marcar um gol em uma partida.

Sendo assim, este modelo segue os mesmos princípios do modelo anterior, entretanto, usando toda a base de dados disponível.

In [ ]:
# Importação de bibliotecas necessárias para o funcionamento do código
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
import numpy as np
import sklearn as sk
from sklearn.cluster import KMeans
from sklearn.metrics import confusion_matrix

In [ ]:
# Abertura das bases de dados
df = pd.read_csv('/content/drive/Shareddrives/Drive IBMatch/Data/new_teams.csv')
df['team_name'] = df['common_name_x']
df = df.drop(columns=['common_name_x'])

matches = pd.read_csv('/content/drive/Shareddrives/Drive IBMatch/Data/brazil-serie-a-matches-2024-to-2024-stats (5).csv', sep=";")
matches = matches[['home_team_name', 'away_team_name', 'home_team_goal_count', 'away_team_goal_count','home_team_goal_timings', 'away_team_goal_timings','status']]

O trecho a seguir ajusta os nomes dos times para evitar problemas relacionados a compatibilidade.

In [ ]:
def check_names(name1, name2):
    # Comparar se um nome está contido no outro (ignora maiúsculas/minúsculas)
    name1_lower = name1.lower()
    name2_lower = name2.lower()
    return name1_lower in name2_lower or name2_lower in name1_lower

# Iterar pelas linhas do segundo DataFrame e comparar com o primeiro
def adjust_names(matches_column):
  for idx2, name2 in matches[matches_column].items():
      for name1 in df['team_name']:
          if check_names(name1, name2):
              matches.at[idx2, matches_column] = name1
              break

adjust_names('home_team_name')
adjust_names('away_team_name')

In [ ]:
alt = df.drop(columns=df.select_dtypes(include='object')) # cria variável alternativa para análises posteriores

O trecho a seguir plota um gráfico cotovelo para determinação da melhor quantidade de clusters para o K-means. A seleção da quantidade ideal se dá pela análise de onde ocorre uma quebra de continuidade na linha, caracterizada pela mudança repentina no angulo formado por dois segmentos (cotovelo).

In [ ]:
wcss = []
for i in range(1, 11):
    kmeans = KMeans(n_clusters = i, init = 'k-means++', n_init='auto', random_state=43)
    kmeans.fit(alt)
    wcss.append(kmeans.inertia_)

px.line(x=range(1, 11), y=wcss)

A partir do gráfico anterior, aplicou-se o algoritmo K-means para agrupamentos dos times escolhendo como quantidade de clusters 4.

In [ ]:
kmeans_plain = KMeans(n_clusters=4, n_init='auto', random_state=43)
kmeans_plain.fit(alt)
kmeans_plain.labels_ = kmeans_plain.labels_.astype(str)

kmeans_plain.labels_

Buscou-se, partir disso, compreender a importância de cada variável para a a formação dos _clusters_. Para isso, utilizou-se de uma estrátégia na qual é comparada a variação da inércia (medida da dispersão dos elementos de um grupo em relação ao centróide) do sistema ao se remover um de seus atributos. Nesse sentido, podemos averiguar qual das _features_ são mais significativas no agrupamento dos elementos.

In [ ]:
scaler = StandardScaler()
alt_scaled = scaler.fit_transform(alt)

kmeans_plain = KMeans(n_clusters=4, n_init='auto', random_state=43)
kmeans_plain.fit(alt_scaled)

def calcular_inercia(X):
    kmeans = KMeans(n_clusters=4, n_init='auto', random_state=43)
    kmeans.fit(X)
    return kmeans.inertia_

inercia_total = calcular_inercia(alt_scaled)

importancias = []
for i in range(alt_scaled.shape[1]):
    alt_reduced = np.delete(alt_scaled, i, axis=1)
    inercia_reduced = calcular_inercia(alt_reduced)
    importancias.append(inercia_total - inercia_reduced)

importancias_df = pd.DataFrame({
    'feature': alt.columns,
    'importance': importancias
}).sort_values(by='importance', ascending=False)

fig = px.bar(importancias_df,
             x='feature',
             y='importance',
             title='Importância das Features no KMeans',
             labels={'importance': 'Redução de Inércia', 'feature': 'Feature'})

fig.show()


Visto que, para aplicação do K-means, foi necessária a remoção de informações categóricas dos dados, o trecho a segui recoloca essas informações na tabela para análises posteriores.

In [ ]:
alt = pd.concat([pd.DataFrame(alt), pd.DataFrame(df['team_name'])], axis=1)
alt['Class'] = pd.Series(kmeans_plain.labels_.astype(str), name="Class")
alt.head(3)

Exposição de gráfico de dispersão que relaciona o total de primeiros gols feitos pelos times e a quantidade total de gols para averiguar o resultado da clusterização.

In [ ]:
px.scatter(alt, x=alt.iloc[:, 3], y=alt.iloc[:, 4], color=alt['Class'],
           title="Relação entre a quantidade de primeiros gols em partidas e o total de gols feitos",
           labels={'x':'Quantidade de primeiros gols em partidas', 'y':'Total de gols feitos','Class':'Grupo de times'})

Exposição de gráfico de dispersão que relaciona o total de primeiros gols feitos pelos times, a quantidade total de gols feitos e o total de gosl tomados para averiguar o resultado da clusterização.

In [ ]:
# Plota gráfico tridimensional dos pontos
fig = px.scatter_3d(x=alt.iloc[:, 3],
                    y=alt.iloc[:, 4],
                    z=alt.iloc[:,5],
                    color=alt['Class'],
                    title='Relação entre a quantidade de primeiros gols em partidas, total de gols feitos e total de gols tomados',
                    labels={'x':'Quantidade de primeiros gols em partidas', 'y':'Total de gols feitos', 'z':'Total de gols tomados', 'color':'Grupo de times'})
fig.show()

Com os clusters estabelecidos, realizou-se a determinação da probabilidade de que um time seja o primeiro a marcar um gol contra determinado adversário. Para isso, foi necessário o estabelecimento de alguma funções, descritas a seguir:
- `first_goals_count`: realiza a contagem de quantas vezes um time foi o responsável pelo primeiro gol de uma partida contra os times do cluster do seu adversário;
- `return_goal_probability`: retorna a probabilidade de um time ser o primeiro a marcar um gol contra um determinado adversário usando probabilidade empírica.

In [ ]:
#
def first_goals_count(team_name, opponent_team_class):
  team_first_goals_count = 0
  for i in matches.iterrows():
    min_target = 95
    min_opponent = 95
    if i[1]['home_team_name'] == team_name and opponent_team_class.isin([i[1]['away_team_name']]).any():
      if int(i[1]['home_team_goal_count']) != 0:
        try:
          min_target = int(str(i[1]['home_team_goal_timings']).split(',')[0].replace("'", '')[:2])
        except ValueError:
          min_target = 95

      if int(i[1]['away_team_goal_count']) != 0:
        try:
          min_opponent = int(str(i[1]['away_team_goal_timings']).split(',')[0].replace("'", '')[:2])
        except ValueError:
          min_opponent = 95

    elif i[1]['away_team_name'] == team_name and opponent_team_class.isin([i[1]['home_team_name']]).any():
      if int(i[1]['away_team_goal_count']) != 0:
        try:
          min_target = int(str(i[1]['away_team_goal_timings']).split(',')[0].replace("'", '')[:2])
        except ValueError:
          min_target = 95

      if int(i[1]['home_team_goal_count']) != 0:
        try:
          min_opponent = int(str(i[1]['home_team_goal_timings']).split(',')[0].replace("'", '')[:2])
        except ValueError:
          min_opponent = 95

    if min_target < min_opponent:
      team_first_goals_count += 1

  return team_first_goals_count

def return_goal_probability(first_team, second_team, data):
  first_team_class = data.loc[data['team_name'] == first_team, 'Class']
  second_team_class = data.loc[data['team_name'] == second_team, 'Class']

  if first_team_class.empty or second_team_class.empty:
    return (0, 0)

  first_class_teams = data.loc[(data['Class'] == first_team_class.iloc[0]), 'team_name']
  second_class_teams = data.loc[(data['Class'] == second_team_class.iloc[0]), 'team_name']

  first_team_fg = first_goals_count(first_team, second_class_teams)
  second_team_fg = first_goals_count(second_team, first_class_teams)

  first_team_goal_status = first_team_fg/matches[(matches['status'] == 'complete') & (((matches['home_team_name'] == first_team) & (matches['away_team_name'].isin(second_class_teams).any())) | ((matches['away_team_name'] == first_team) & (matches['home_team_name'].isin(second_class_teams).any())))]['home_team_name'].count()
  second_team_goal_status = second_team_fg/matches[(matches['status'] == 'complete') & (((matches['home_team_name'] == second_team) & (matches['away_team_name'].isin(first_class_teams).any())) | ((matches['away_team_name'] == second_team) & (matches['home_team_name'].isin(first_class_teams).any())))]['home_team_name'].count()

  first_team_goal_probability = first_team_goal_status/(first_team_goal_status+second_team_goal_status)
  second_team_goal_probability = second_team_goal_status/(first_team_goal_status+second_team_goal_status)

  return (first_team_goal_probability, second_team_goal_probability)

Para testar a eficácia das funções anteriores e do modelo como um todo, foram estabelecidas outras funções auxiliares, descritas a seguir:
- `first_gol`: dada uma partida, indica quem foi o primeiro a marcar gol
- `define_previsions`: dada uma partida, indica qual time tem a maior probabilidade de ser o primeiro a marcar o primeiro gol com base no modelo descrito anteriormente;
- `give_results`: compara o resultado previsto com aquele que ocorreu de fato, retornando "Acerto", casos os dois sejam iguais, "Erro" caso sejam diferentes e "Indeterminado", caso não tenham havido gols na partida.

In [ ]:
def first_goal(row):
  # Função para limpar e converter os tempos dos gols para float
  def convert_goal_timings(goal_timings):
      if pd.isna(goal_timings) or not goal_timings:
          return []
      # Remove o apóstrofo e converte para float
      return [float(time.replace("'", "")) for time in goal_timings.split(',')]

  # Extrai os momentos dos gols como listas de floats, ou lista vazia se não houver gols
  home_goals = convert_goal_timings(row['home_team_goal_timings'])
  away_goals = convert_goal_timings(row['away_team_goal_timings'])

  # Combina os gols em uma lista de tuplas (time, tempo do gol)
  all_goals = [(row['home_team_name'], goal_time) for goal_time in home_goals] + \
              [(row['away_team_name'], goal_time) for goal_time in away_goals]

  # Se não houver gols, não há um primeiro a pontuar
  if not all_goals:
      return None

  # Ordena os gols pelo tempo e retorna o time que fez o primeiro gol
  first_goal = min(all_goals, key=lambda x: x[1])
  return first_goal[0]

In [ ]:
def define_previsions(target, reference):
  target['prevision'] = ''
  for idx, row in target.iterrows():
    prob1 = return_goal_probability(row['home_team_name'], row['away_team_name'], reference)[0]
    prob2 = return_goal_probability(row['home_team_name'], row['away_team_name'], reference)[1]
    if prob1 > prob2:
      target.at[idx, 'prevision'] = row['home_team_name']
    elif prob1 == prob2:
      target.at[idx, 'prevision'] = None
    else:
      target.at[idx, 'prevision'] = row['away_team_name']
  return target

In [ ]:
def give_results(target):
  for idx, row in target.iterrows():
    if row['first_goal'] == row['prevision']:
      target.at[idx, 'result'] = "Acerto"
    elif row['first_goal'] == None:
      target.at[idx, 'result'] = "Indeterminado"
    elif row['first_goal'] != row['prevision']:
      target.at[idx, 'result'] = "Erro"
  return target

Usando das funções descritas anteriormente, aplicou-se o algoritmo sobre toda a base de dados, visando averiguar quanto das partidas o algoritmo foi capaz de prever corretamente, apresentando os resultados nos gráficos a seguir:

In [ ]:
# Aplica os algoritmos anteriores em todas as partidas que já ocorreram
partidas = matches.loc[matches['status']== 'complete', ['home_team_name', 'away_team_name']]
temp = matches.loc[matches['status']== 'complete', ['home_team_name', 'away_team_name', 'away_team_goal_timings', 'home_team_goal_timings']]

partidas['first_goal'] = temp.apply(first_goal, axis=1)
partidas = define_previsions(partidas, alt)
partidas = give_results(partidas)

O gráfico a seguir apresenta, quantitativamente, a distribuição de acertos, erros e indeterminações que o modelo foi capaz de executar.

In [ ]:
px.histogram(partidas, x='result', color='result', category_orders=dict(result=["Acerto", "Erro", "Indeterminado"]), title='Distribuição dos resultados das previsões', labels={'result':'Resultado da previsão', 'count':'Contagem'})

Estabeleu-se também uma comparação de quanto cada resultado corresponde ao total das predições, apresentando-as a seguir:

In [ ]:
px.pie(partidas, names='result', title='Distribuição dos resultados das previsões em porcentagem', labels={'result':'Resultado da previsão'})

Por fim, constuiu-se uma matriz de confusão para averiguar como estavam distribuídas as predições e se havia algum viés no modelo

In [ ]:
df_valid = partidas[partidas['result'] != 'Indeterminado']

y_true = df_valid['first_goal']
y_pred = df_valid['prevision']

y_true = df_valid['first_goal'].fillna('')
y_pred = df_valid['prevision'].fillna('')

cm = confusion_matrix(y_true, y_pred)

labels = np.unique(np.concatenate((y_true, y_pred)))

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=labels, yticklabels=labels)

plt.title('Matriz de Confusão')
plt.xlabel('Previsão')
plt.ylabel('Resultado Real')

plt.show()

## 3. Comparação de resultados e métricas

A partir da apresentação dos modelos, podemos comparar as métricas atingidas e os resultados alcançados a fim de determinar qual está desempenhando melhor a tarefa de predizer qual time será o primeiro a marcar gol em uma partida.

A priori, quando averiguamos a taxa de acerto de ambas as abordagens, nota-se que o segundo modelo atingiu uma taxa de acerto de 69,8%, sendo apenas 0,5 pontos percentuais maior que o primeiro modelo (69,3%). Nesse sentido, observa-se que, apesar da seleção de _features_ tendo ocorrido de maneira distinta entre os modelos, ambos atingiram resultados muito próximos.

Nesse sentido, quando comparamos os atributos selecionados pelo primeiro modelo, percebemos que foi benéfica a remoção de alguns atributos, visto que a taxa de acertos continuou elevada mesmo com os dados contendo menos atributos.

Observa-se, também, por meio das matrizes de confusão, que os modelos estão com uma distribuição de predições relativamente homogênea, indicando que não há um viés proeminente para determinado time ou resultado. Isso se deve ao fato dos algoritmos desconsiderarem informações momentâneas, como o fato de um time estar jogando em casa ou ser visitante, que podem afetar o resultado final, seja aumentando a acurácia ou criando viéses nas predições.

A presença de valores indeterminados no segundo modelo se apresenta como um ponto de atenção, visto que sua ausência no primeiro modelo pode revelar que uma parcela dos erros do modelo sejam, na verdade, partidas sobre as quais não foi possível realizar uma predição devido ao fato de não terem ocorridos gols.



## 4. Conclusões

Por meio das análises anteriores, observa-se que os dois modelos apresentaram resultados muito semelhantes, apesar de terem utilizado de bases de dados distintas.

A ausência de um filtro de valores indeterminados no primeiro modelo é um ponto de atenção, visto que pode "inflar" a taxa de erro ao não diferenciá-los. Nesse sentido, o correto tratamento desses valores deve ser levado em consideração em futuras modificações.

Ademais, a semelhança nos resultados de ambos os modelos, mesmo com o primeiro tendo usado menos atributos nas suas análises, levanta outro ponto de atenção em relação a potêncial presença excesso de dimensões no modelo, sendo necessários testes removendo alguns atributos para averiguar seu impacto na taxas de acerto, erro e indeterminação.

Por fim, o uso de um modelo de probabilidade empírica apresenta vantagem ao fornecer grande explicabilidade aos algoritmos, permitindo que sejam feitas modificações pontuais de maneira facilitada em relação ao uso de outros algortimos e abrindo espaço para a realização das melhorias citadas anteriormente. De modo geral, ambos os modelos foram capazes de desempenhar de maneira satisfatória a tarefa de predizer os time responsável pelo primeiro ponto nas partidas.